# Data Governance Analysis
This notebook runs all governance checks and saves results to a Delta table

In [ ]:
# Create widgets for parameters
dbutils.widgets.text("catalog_name", "main", "Catalog Name")
dbutils.widgets.text("schema_name", "default", "Schema Name")

# Get parameter values
catalog_name = dbutils.widgets.get("catalog_name")
schema_name = dbutils.widgets.get("schema_name")

print(f"Using catalog: {catalog_name}")
print(f"Using schema: {schema_name}")

In [ ]:
import governance_analyzer as ga
from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType

In [ ]:
# Define governance checks
checks = [
    ("Metastore setup", "Connect a Metastore to your Workspace", ga.check_metastore_connected),
    ("Metastore setup", "The workspace is in the same region as the metastore", ga.check_metastore_region),
    ("Identity", "Use SCIM or AIM from an Identity Provider", ga.check_scim_aim_provisioning),
    ("Identity", "Account Admin role is assigned to a group", ga.check_account_admin_group),
    ("Identity", "Metastore Admin role is assigned to a group", ga.check_metastore_admin_group),
    ("Identity", "Workspace Admin role is assigned to a group", ga.check_workspace_admin_group),
    ("Identity", "Catalog Admin role is assigned to a group", ga.check_catalog_admin_group),
    ("Identity", "At least 1 user is an account admin", ga.check_at_least_one_account_admin),
    ("Identity", "Less than 5% of users are Account Admin", ga.check_account_admin_percentage),
    ("Managed Storage", "Create multiple Catalogs based on environment/BU/team", ga.check_multiple_catalogs),
    ("Managed Storage", "No Catalog is bound to all workspaces", ga.check_catalog_binding),
    ("Managed Storage", "Use Managed tables and volumes > 70%", ga.check_managed_tables_percentage),
    ("Managed Storage", "No ADLS or S3 buckets outside UC", ga.check_no_external_storage),
    ("Managed Storage", "No external volumes/tables at external location root", ga.check_external_location_root),
    ("Managed Storage", "Independent storage credentials per external location", ga.check_storage_credentials),
    ("Compute/Cluster Policy", "Compute is UC activated with right access mode", ga.check_uc_compute),
    ("Migration Completeness", "No data in hive metastore", ga.check_no_hive_data),
    ("Migration Completeness", "Hive metastore is disabled", ga.check_hive_disabled),
    ("Migration Completeness", "0 mount storage accounts to DBFS", ga.check_no_dbfs_mounts),
    ("Audit & Lineage Coverage", "All system tables activated (70%)", ga.check_system_tables),
    ("Audit & Lineage Coverage", "70% of managed tables have predictive optimization", ga.check_predictive_optimization),
    ("Audit & Lineage Coverage", "Data quality activated on 50% of tables", ga.check_data_quality),
    ("Privileges", "Production jobs use service principals", ga.check_service_principals),
    ("Privileges", "Modify access to production is limited", ga.check_production_access),
    ("Privileges", "70% of assets have groups as owners", ga.check_group_ownership),
]

In [ ]:
# Run all checks
results = []
timestamp = datetime.now()

for category, task_name, check_func in checks:
    result = check_func()
    results.append({
        "timestamp": timestamp,
        "category": category,
        "task_name": task_name,
        "status": result["status"],
        "score": result["score"],
        "details": result["details"]
    })

print(f"Completed {len(results)} governance checks")

In [ ]:
# Convert to DataFrame
df = spark.createDataFrame(results)
display(df)

In [ ]:
# Save to Delta table
table_name = "governance_results"
full_table_name = f"{catalog_name}.{schema_name}.{table_name}"

df.write \
    .format("delta") \
    .mode("append") \
    .saveAsTable(full_table_name)

print(f"Results saved to {full_table_name}")

In [ ]:
# Calculate summary statistics
summary_df = spark.sql(f"""
    SELECT 
        category,
        COUNT(*) as total_checks,
        SUM(CASE WHEN status = 'pass' THEN 1 ELSE 0 END) as passed_checks,
        SUM(score) as total_score,
        ROUND(SUM(CASE WHEN status = 'pass' THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) as pass_rate
    FROM {full_table_name}
    WHERE timestamp = (SELECT MAX(timestamp) FROM {full_table_name})
    GROUP BY category
    ORDER BY category
""")

display(summary_df)

In [ ]:
# Overall governance score
overall_df = spark.sql(f"""
    SELECT 
        COUNT(*) as total_checks,
        SUM(CASE WHEN status = 'pass' THEN 1 ELSE 0 END) as passed_checks,
        SUM(score) as achieved_score,
        ROUND(SUM(CASE WHEN status = 'pass' THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) as governance_score
    FROM {full_table_name}
    WHERE timestamp = (SELECT MAX(timestamp) FROM {full_table_name})
""")

display(overall_df)

## Dashboard Creation

To create a dashboard in Databricks:

1. Go to **SQL Workspace** in Databricks
2. Create a new **SQL Query** for each visualization:
   - **Governance Score by Category**: Use the summary query above
   - **Overall Score**: Use the overall query above
   - **Failed Checks**: Query for failed checks
3. Create **Visualizations** for each query (bar charts, counters, tables)
4. Create a new **Dashboard** and add all visualizations
5. Set up **refresh schedule** if needed